# Command-Based Programming

We've seen the simpler building blocks of command-based programming in the previous document--how we can get inputs and group devices into subsystems--as well as how projects are generally organized.

Now we'll take a close look at the thing that gives `Command-Based Programming` its name: Commands, as well as some of the features which tie very closely into the command-based framework, and BooleanEvents/Triggers. This section will begin, however, with a discussion about a concept with many names, such as `lambdas`, `closures`, `anonymous-functions`. They present an alternative method for the representation of actions in our projects and are crucial towards the new kind of command-based programming that is being pushed by the designers of WPILib.

As such, it will be worthwhile to look at them in great (albeit tangentially related) detail, and to really understand their functionality.

As you go through this first section, it is recommended to give everything here a try yourself in JShell. Try replicating the results from the codeblocks and see what results, play around with creating and applying them and see if their behavior matches your expections. If not, try and figure out why, and don't hesistate to ask if something is especially confusing. Lambdas, and their related features are famously a source of confusions for many java programmers. 

## Functions (a continuation)

In the first two documents, we covered 'functions' in the form of:

In [1]:
int square(int x){
    return x*x;
}


before revealing that in fact, these are just `method`s in disguise.


### The Shortfalls of Methods

At a glance, this distinction seems to have little to no consequence -- so what if our functions are methods of classes instead of actual function values? What difference would that make? 

The issue with just using methods is that methods can't be passed as values, like an integer or String. This presents a problem whenever we want to have the ability to perform any valid but arbitrary action, instead of some predetermined action.

That is, lets say we have an array, and we want to map some operation over all of its values. Instead of using some particular method like `square`:

In [2]:
int[] map(int[] xs){
    int[] newArr = new int[xs.length];
    // appender (i++) or (++i) is a shorter and more popular alternative to the previous (i += 1)
    for (int i = 0; i < xs.length; i++){ 
        newArr[i] = square(xs[i]);
    }
    return newArr;
}

System.out.println(Arrays.toString(map(new int[]{1,2,3,4})));

[1, 4, 9, 16]


But what if we want some way to use any valid function that takes an integer, and returns another integer:

```java
int[] map(int[] xs, ??? f){ // what could this type be??
    int[] newArr = new int[xs.length];
    for (int i = 0; i < xs.length; i++){ 
        newArr[i] = f(xs[i]);
    }
    return newArr;
}
```

As it stands, there is no way to directly pass a method, even a valid one, in such a manner. So instead, we must compromise and pass an object of some sort that can represent a function.

### Functional Interfaces 


But what shape would such a object take? And how does that 'representation' of a function work? 

Such a function would have to behave analogously to a method when it comes to its usage, being able to be applied in much the same way a method would be called. Beyond that however, it doesn't need anything else -- neither any fields or other methods.

In this case, we can simply represent any particular function as a class with only one method:


In [3]:
class square {
    public int apply(int x) {
        return x * x;
    }
}

new square().apply(5);

25

But that only works for particular functions, where we know in advance the implementation. What if we wanted to create something that encompasses all possible functions of a certain signature, like functions similar to `square` which only take one integer as input and give an integer as its output?

For that, we can't know how our objects' single method might be implemented, since each function could have very distinct implementations. As such, we can represent such functions as an interface with one unimplemented method:

In [4]:
interface MyFunction {
    int apply(int x);
}

int[] map(int[] xs, MyFunction f){
    int[] newArr = new int[xs.length];
    for (int i = 0; i < xs.length; i++){ 
        // calling our 'function'
        newArr[i] = f.apply(xs[i]);
    }
    return newArr;
}

then to create an instance of our function, we would implement this interface and provide an implementation for this method:

In [5]:
class cube implements MyFunction{
    public int apply(int x) {
        return x * x * x;
    }
}

System.out.println(
    Arrays.toString(
        map(
            new int[]{1,2,3,4},
            new cube() // passing an object that represents our function
        )
    )
);

[1, 8, 27, 64]


The `MyFunction` interface is an example of what is known as a `SAM (Single Abstract Method) type`, or a `Functional Interface`, since the interface only contains a single abstract method, a fact which will become very important in the next section. It could have any number of static, private, or default methods -- but so long as it only has one abstract method, it is a valid functional interface.

In [6]:
interface NonFunctional { // 2 abstract methods
    double foo(double x);
    int bar(int x);
}

interface Functional {
    double foo(double x);
}

### Anonymous Functions - The Lambda

One thing you might've noticed about this pattern of
- creating an interface which represents a type of function
- creating a class that implements the interface to implement a particular function
- creating an instance of the class get a function value.

is that it's particularly repetitive to do on it own. Luckily, there are existing constructs in place that help ease that process a bit. For the interface creation, we'll see some existing examples in the java standard library later on. However for the latter steps of creating a class/instance, java provides a core language feature called the `lambda expression`.

#### Lambdas

Lambda expressions, whose name comes from the related [lambda calculus](https://en.wikipedia.org/wiki/Lambda_calculus)(though knowledge of it won't be necessary here), are a syntactical feature allowing for easier construction of these function values. They are structured [as follows](https://docs.oracle.com/javase/specs/jls/se20/html/jls-15.html#jls-15.27):

`(parameter_list) -> lambda_body`

```java
// e.g.
(int x) -> x * 2;
(int x) -> {
    int y = x*2;
    return y;
};
```

where a parameter list containing all the parameters of the function is followed by an arrow pointing towards the body of the lambda.



The structure of a lambda expression mirrors that of a method definition. The format for the parameter list of both are identical, and the lambda_body is analogous to the method body. The only difference being that if the lambda_body is just one expression, an explicit `return` isn't necessary. 

### Existing Examples

An example already present in java would be the [`Comparator<T>`](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/Comparator.html) interface, whose body contains one abstract method (amid a medley of static methods): 

```java
// intended behavior for compare:
// if compare(o1, o2) is:
//     < 0 then o1 < o2 
//     = 0 then o1 = o2
//     > 0 then o1 > o2
interface Comparator<T>{
    int compare(T o1, T o2);
}
```

which can then be implemented with the following lambda functions:

In [7]:
// standard order comparator for numbers
Comparator<Integer> cmp = (i, j) -> i - j;
System.out.println(cmp.compare(2, 1));
// reverse order comparator for strings
Comparator<String> cmp2 = (s1, s2) -> s2.compareTo(s1);
System.out.println(cmp2.compare("book", "barn"));

1
-14


we could then use our comparators (or existing comparators) with java's library to do things like sort a list

In [8]:
ArrayList<String> xs = new ArrayList<String>();
xs.addAll(List.of(
    "book", "barn", "apple", "banana", "orange", "pear"
));
System.out.println(xs);
xs.sort(cmp2);
System.out.println("now in reverse order:");
System.out.println(xs); 

[book, barn, apple, banana, orange, pear]
now in reverse order:
[pear, orange, book, barn, banana, apple]


### Custom Examples

we could do the same with our `MyFunction` interface from above, and define `Cube` using a short lambda expression instead of a whole class definition, like so:

In [9]:
MyFunction cube = x -> x * x * x;
// reusing our map function
int[] res = map(new int[]{1,2,3,4}, cube);
System.out.println(
    Arrays.toString(res)
);


[1, 8, 27, 64]


which looks quite a bit cleaner than our definition before.

#### UserAnalog and UserDigital

looking at our previous document, we can see another example of lambdas in use: that being our `UserAnalog` and`UserDigital` interfaces.

Looking at their definition, they have the structure as a functional interface, with a singular abstract instance method (and some static helper methods). As such, we can use lambdas to create instances of them, and use them in our code:

In [10]:
// for analog inputs (e.g. joysticks or triggers)
public interface UserAnalog {
    // ... helper method(s)
    // the singular abstract (instance) method
    public double get(); 
}
// for digital inputs (e.g. buttons)
public interface UserDigital {
    // ... helper method(s)
    // the singular abstract (instance) method
    public boolean get(); // note this returns a boolean instead of a double 
}
UserAnalog ua = () -> 3.14;
UserDigital ud = () -> true;
System.out.println(ua.get());
System.out.println(ud.get());

3.14
true


### Method References

Beyond lambda expressions, java also provides a contract that allows the creation of references to existing methods. For instance, instead of capitalizing an array of strings using our previous `map` function by providing the lambda expression `(int i) -> Math.abs(i)`, we can create a static method reference for `Math.abs` by using two colons: 

In [11]:
int[] res = map(new int[]{-2,-1,0,1,2}, Math::abs);
System.out.println(
    Arrays.toString(res)
);

[2, 1, 0, 1, 2]


method references can also be used to create references to instance methods, by using the instance name instead of the class name:

In [12]:
class Num{
    int x;
    Num(int x){
        this.x = x;
    }
    int add(int y){
        return x + y;
    }
}
var five = new Num(5);
int [] ret = map(new int[]{1,2,3,4}, five::add);
System.out.println(
    Arrays.toString(ret)
);

[6, 7, 8, 9]


## An Application: BooleanEvents 

one way we can use lambdas is to listen for certain events that we then connect with actions. For instance, earlier we introduced a more primitive method of directly polling with our Controller methods and our `UserDigital` and `UserAnalog` interfaces. 

### Limits of Polling Boolean Inputs

While the aforementioned method works well for analog inputs and button-*holding*. It's rather ineffective at listening for button-*pressing* and *releasing*, which involve listening multiple times for changes from `down -> up` or `down -> up`. 

Furthermore, if we want to bind combinations of button presses to an event (e.g. pressing `A` and `B` at the same time to perform a pre-assigned action), the complexity of the checks grows quite quickly.  


### [EventLoop](https://github.wpilib.org/allwpilib/docs/release/java/edu/wpi/first/wpilibj/event/EventLoop.html) and [BooleanEvent](https://github.wpilib.org/allwpilib/docs/release/java/edu/wpi/first/wpilibj/event/BooleanEvent.html) Objects

luckily WIPLib supplies us two tools to help with this task. those being the `BooleanEvent` and `EventLoop`. 

In [13]:
// 6 lines below are just for setting up the environment in this particular document
// note that this written with 2023 version, so should be updated accordingly in future years if used
%mavenRepo wpilib https://frcmaven.wpi.edu/artifactory/release
%mavenRepo ctre https://maven.ctr-electronics.com/release
%maven edu.wpi.first.wpilibj:wpilibj-java:2023.4.3
%maven edu.wpi.first.wpiutil:wpiutil-java:2023.4.3
%maven edu.wpi.first.wpimath:wpimath-java:2023.4.3
%maven edu.wpi.first.wpilibNewCommands:wpilibNewCommands-java:2023.4.3
// necessary imports
import edu.wpi.first.wpilibj.event.EventLoop;
import edu.wpi.first.wpilibj.event.BooleanEvent;
// create boolean event with an event loop
var loop = new EventLoop();
var event = new BooleanEvent(loop, () -> true);
// a demonstration of a more dynamic event
var flag = false;
var event2 = new BooleanEvent(loop, () -> flag);


where the EventLoop `loop` acts as the core manager or coordinator of all the boolean events (`event` and `event2`) that we created, and the event itself is an association between that loop and a function (or predicate) that checks whether some condition it fulfilled. This very general description allows it to be used in an assortment of relevant cases, from the aforementioned button presses, to checking if a sensor has reached a certain value, or a combination of both (say pressing a button while a motor has spun up to a desired speed and the robot is oriented to a certain angle). 

Outside of just associating an event with its condition, the event also allows us to specify what action should be taken when the condition is fulfilled. This is done by passing in another function which does not take or return any values. 

In [14]:
// bind certain actions to the events when their condition is met
event.ifHigh(() -> System.out.println("alwaying running"));
event2.ifHigh(() -> System.out.println("flag on"));

Futhermore, the BooleanEvent class also gives us the tools to construct more complex events from existing ones. Especially important is the ability to listen for *changes* in the condition, instead of just to the condition itself, which can be achieved with the `rising` and `falling` methods, which construct a new event that only fires when the condition changes from `false -> true` or `true -> false` respectively.


In [15]:
// we can also string events together to create more complex events
var enabled = event2.rising();
var disabled = event2.falling();

enabled.ifHigh(() -> System.out.println("flag enabled"));
disabled.ifHigh(() -> System.out.println("flag disabled"));


To then query the status of all of the events (or `poll` them), we'd call the EventLoop's `poll` method, which checks the conditions for all associated events and if they are true, executes the associated action.

In [16]:
// poll to invoke actions
loop.poll();
System.out.println("-".repeat(16));
// set flag and try again
flag = true;
loop.poll(); 
System.out.println("-".repeat(16));
// ... and again
flag = false;
loop.poll();
System.out.println("-".repeat(16));
// ... and once more
loop.poll();

alwaying running
----------------
alwaying running
flag on
flag enabled
----------------
alwaying running
flag disabled
----------------


alwaying running


In practice, this would most likely be called in one of the `periodic` methods of our Robot class in order to avoid missing any event activations. 

```java
@Override 
public void teleopPeriodic(){
    // other code...
    loop.poll();
}
```

Events can also be constructed using logical operations (and, or, not) on existing events, which allows us to create more complex events from simpler ones. For instance, if we had event `a` for button `A` being held, and event `b` for button `B` being held, we could create an event `c` for when only one of is `A` and `B` being pressed with the following snippet: 
    
```java
// a && !b || !a && b (aka a xor b but alas no xor method)
var c = a.and(b.negate()).or(a.negate().and(b));
```

> Note: in practice, many events that are prone to rapid changes (such as button presses) should be filtered with the `debounce` method, which prevents the event from unwantingly firing multiple times in a short period and repeatedly triggering the associated action.

> Note2: chaining the rising and falling for the same event in order to listen for changes in the condition might not work as expected, at least not in the demonstration environment, perhaps due to their measurement changing the internal state of the event. As such, consider attaching the same action for the seperate rising and falling events. For any maintainer reading this, check if adding a debounce rectified this issue.

## Commands 

> represent actions the robot can take. Commands run when scheduled, until they are interrupted or their end condition is met. Commands are very recursively composable: commands can be composed to accomplish more-complicated tasks

Commands form the second important abstraction in the command-based paradigm and are the backbone for implementing more complex robot behaviors. Whereas subsystems are simple groupings of individual hardware devices that work together as one unit, Commands allow for the modeling of more sophisticated, high level actions more akin to how we might describe them in natural language.


### A Basic Structure / The Manual Way 

Commands, like described in the text above, run when scheduled, until they are interrupted or their end condition is met, and so primarily consist of four core methods to facilitate that workflow: 
1. `initialize` - called once when the command is first scheduled and starts execution
    - where any necessary setup should be done (similar to the `Init` methods in `Robot`)
2. `execute` - called repeatedly while the command is scheduled
    - where the bulk of the command's logic should be placed (similar to the `Periodic` methods in `Robot`)
3. `isFinished` - called repeatedly while the command is scheduled
    - determines whether the objective of the command has been met
4. `end` - called once when the command is finished to do any necessary cleanup   
    - method has an `interrupted` parameter that describes whether it finished successfully (`interrupted=false`) or was shut down by the scheduler (`interrupted=true`)

Commands can be manually created by extending the `CommandBase` class, the template for creating arbitrary commands, and overriding the aforementioned methods to achieve the desired behavior. 

In CommandBase, the default behavior for each `initalize`, `execute`, and `end` is to do nothing, and the default behavior for `isFinished` is to return `false`, which means that the default command will run until the robot is disabled and will do nothing. 
 
 


In [17]:
// disclaimer: this was written with 2023 version, so should be updated accordingly in future years if used
// in particular, CommandBase will be deprecated, and the new Command class will take over its function
// at present, Command is an interface, not a class, so CommandBase is used here instead
import edu.wpi.first.wpilibj2.command.CommandBase;

class ExampleCommand extends CommandBase {
    @Override
    public void initialize() {
        System.out.println("initializing");
    }

    @Override
    public void execute() {
        System.out.println("executing");
    }

    @Override
    public void end(boolean interrupted) {
        if (interrupted) {
            System.out.println("interrupted");
        } else {
            System.out.println("ending");
        }
    }
}


### Managing Requirements

A Command like the one constructed above isn't very helpful on its own, commands often need to interact with subsystems to perform their desired task. To specify which subsystems are required, we need to call the `addRequirements` method of `CommandBase` on all the subsystems that we need.

Specifying which requirements a command has is important for the scheduler (thing that manages execution of our commands) to know in order to avoid conflicts, such as a case where two commands are try to use the same subsystem at the same time but with differing inputs. 

In the case of these manually defined events, this is often done in the constructor where the relevant subsytems are passed in as parameters:

> Aside: this is an example of a pattern called `dependency injection` where things that the command needs (dependencies) are provided (injected). Here, this pattern helps seperate the creation and usage of our subsystems


In [18]:
import edu.wpi.first.wpilibj2.command.SubsystemBase;
import edu.wpi.first.wpilibj2.command.CommandBase;

class ExampleSubsystem extends SubsystemBase {
    // subsystem code here
}

class ExampleCommand extends CommandBase {
    public ExampleCommand(ExampleSubsystem subsys1, ExampleSubsystem subsys2){
        this.addRequirements(subsys1, subsys2); 
    }
    @Override
    public void execute() {
        // do stuff
    }
}

#### Default Commands

One particular usage for commands command usage for a command is to have it always run on one or more specific subsystems. These commands often handle core background tasks that are essential to the function of the robot, like a `drive` command that controls robot motion.

When the subsystem is not being used by another command, the specified default command will run.

```java
// complete but simplified example of a manually defined command that acts 
public class ClimberCommand extends CommandBase {
    private final Climber climber;
    private final UserAnalog speed;
    // climber is the subsystem which will default to this command
    public ClimberCommand(Climber climber, UserAnalog speed) {
        this.climber = climber;
        this.speed = speed;
        // works b/c addRequirements is a method of CommandBase
        this.addRequirements(climber);
        // making it a default command
        climber.setDefaultCommand(this);
    }

    @Override
    public void execute() {
        climber.extend(speed.get(), false);
    }
}
```

### [The Command Scheduler](https://docs.wpilib.org/en/stable/docs/software/commandbased/command-scheduler.html)

Now that we have our command, how do we run it? We defined methods for what to do in different cases with `initalize`, `execute`, and `end`, but we never called them anywhere.

In the command-based framework, the construct that handles this, as well as the general logic of commands for us is the `CommandScheduler`. The job of the scheduler here is similar to the `EventLoop` from the previous section about `BooleanEvent`s, in that it manages every aspect of the execution of commands like an `EventLoop` manages its events and their associated actions.

Like how the EventLoop must be `poll`ed regularly, so too must the CommandScheduler be `run` regularly. This is done by calling the `run` method of the scheduler in the `robotPeriodic` method of our Robot class.

However, one important difference is that there is only one copy of the CommandScheduler, and it is a static field of the `CommandScheduler` class. As such, we don't need to create an instance of it, and can instead just call the `getInstance` method to get the scheduler.


```java
import edu.wpi.first.wpilibj.TimedRobot;
import edu.wpi.first.wpilibj2.command.CommandScheduler;
public class Robot extends TimedRobot {
    @Override
    public void robotPeriodic(){
        // this specific pattern is called the 'singleton' pattern
        CommandScheduler.getInstance().run();
    }
}
```

To directly add our commands to the scheduler, the `schedule` method of the command can be used:

```java
var exCmd = new ExampleCommand(subsys1, subsys2);
exCnd.schedule();
```

### A More Concise Way of Creating Commands

While the manual way of defining commands works (and which we used the past few years), it's rather verbose and repetitive in its structure, such it's usage of constructor parameters to pass in the required subsystems and inputs. 

Looking at the above ClimberCommand, we can see this is the case, as its fields serve only to give the methods access to the inputs and subsystems without housing any other internal state.

In cases like this where we don't need to modify any internal state, we are able to create lambdas for each of the four (`initialize`, `execute`, `end`, and `isFinished`) methods and create a `FunctionalCommand` by providing by providing those lambdas and any required subsystems to its constructor.

```java
import edu.wpi.first.wpilibj2.command.FunctionalCommand;
// ... in RobotContainer for instance:
Climber climber = ...;
UserAnalog speed = ...;
var climberCommand =  new FunctionalCommand(
    // initialize
    () -> {},
    // execute
    () -> climber.extend(speed.get(), false),
    // end
    interrupt -> {},
    // isFinished
    () -> false,
    // requirements
    climber
)
```

In addition to `FunctionalCommand`, there are also a collection of additional [preset command classes](https://docs.wpilib.org/en/stable/docs/software/commandbased/commands.html#included-command-types) that can be used to create commands for other common and niche use cases. Most of these are also accessible through corresponding factory methods, with the idea of representing the four behaviors .

## Command Factories

The bulk of the useful factory methods for commands are found in the [`Commands`](https://github.wpilib.org/allwpilib/docs/release/java/edu/wpi/first/wpilibj2/command/Commands.html) class as static methods where the particular inputs for each factory can be found in the above link.

### General Command Factories

These command factories usually take one or more functions that describe the behaviors (`initalize`, `execute`,  `isFinished`, and `end`) of the command, as well as any required subsystems. 

The command created by:
- `runOnce(action, reqs...)` runs `action` once and terminates
- `startEnd(start, end, reqs...)`  runs `start` when it starts and `end` when it gets interrupted
- `run(action, reqs...)` runs `action` everytime until interrupted 
-  `runEnd(run, end, reqs... )` runs `run` everytime until interrupted and then runs `end` once

```java
var cmd = Commands.runOnce(() -> System.out.println("Hello World"));
```
> note: if you want to avoid repetition of "Commands." you can use a static import, which will add all static methods of Commands to the namespace of the file. 
> ```java 
> import static edu.wpi.first.wpilibj2.command.Commands.*;
> // ...
> var cmd = runOnce(() -> System.out.println("Hello World"));
> ```


### Timed Commands
a specialized pair of methods whose resulting commands do nothing for a while. These are useful when sequenced with other commands to create a delay between them and wait until conditions are ideal for the next command to run.

Since these commands do nothing, they don't need any subsystems as requirements:

The command created by:
- `waitSeconds(seconds)` - waits `seconds` before finishing
- `waitUntil(condition)` - waits until `condition` is true before finishing




### Combining Commands

the Commands class also includes a collection of methods for combining commands together to create more complex behaviors, such as creating sequences of events, running commands in parallel, and selecting events based on conditions.

Typically these types of combinators take one or more commands and return a new command that represents the combination of the original commands.



#### Sequential Commands

sequential chains of commands can be created with the `sequence` and `repeatingSequence` methods. Both run their commands in order, but `repeatingSequence` will repeat the sequence once it finishes. E.g.:

```java
// should finish in ~3 second when the last print has been finished

var seqcmd = Commands.sequence(
    Commands.runOnce(() -> System.out.println("first")),
    Commands.waitSeconds(1),
    Commands.runOnce(() -> System.out.println("second")),
    Commands.waitSeconds(2),
    Commands.runOnce(() -> System.out.println("third"))
);
```



#### Parallel Commands

parallel commands can be created with the `parallel` and `race` methods, which runs all of its commands at the same time. The event created by `parallel` will finish when all of its commands finish, while the event created by `race` will terminate when any of its commands finish. E.g.:

```java
// should finish in ~2 seconds, when the last print is done
var parcmd = Commands.parallel(
    Commands.runOnce(() -> System.out.println("first")),
    Commands.waitSeconds(1),
    Commands.runOnce(() -> System.out.println("second")),
    Commands.waitSeconds(2),
    Commands.runOnce(() -> System.out.println("third"))
);
```



If we want to use a particular event to determine when a parallel command should terminate, we'd use the `deadline` method, whose event terminates when its first argument finishes. 

#### Selective Commands

The `either` and `select` factories allows for the creation of commands that run one of two commands  or more commands respectively, based on the result of a condition (boolean for `either` and some object for `select`). E.g.:

```java
var cmd1 = Commands.either(
    Commands.runOnce(() -> System.out.println("t")),
    Commands.runOnce(() -> System.out.println("f")),
    () -> true
); // gives a command that prints "t"

var cmd2 = Commands.select(
    Map.of(
        1, Commands.runOnce(() -> System.out.println("one")),
        2, Commands.runOnce(() -> System.out.println("two"))
        4, Commands.runOnce(() -> System.out.println("four"))
    ),
    () -> 4
); // gives a command that prints "four"
```

## Triggers : An Addendum 
With the introduction of Commands, it is also useful to look at an additional construction that we can use to associate commands with events in the same way BooleanEvents work with actions. Here WPILib provides us with the `Trigger` class, which mirrors the `BooleanEvent` class in many ways, but instead of taking in an action as a `Runnable` function, it takes in a `Command` instead. 

The exact set of functionality provided by Triggers can be found in its [javadoc](https://github.wpilib.org/allwpilib/docs/release/java/edu/wpi/first/wpilibj2/command/button/Trigger.html). Some of the method names change between the two classes (such as onFalse and onTrue instead of falling and rising). Also since Commands are more versatile than actions used in BooleanEvents, `Trigger` has some methods which do not have parallels in `BooleanEvent`, such as the while methods and the toggle methods. 

Triggers also allow us to directly link controller inputs to their associated functions, and as a result, (with the exception of default events) much of the scheduling of commands during the teleop phase can be done with Triggers instead of being done manually: E.g.:

```java
// in RobotContainer
var intake = new Intake(); // subsystem
var in = /* get button input (next section) */;
var out = /* get button input (next section) */;
in.whileTrue(intake.suckCommand(1));
out.whileTrue(intake.suckCommand(-1));
```

In addition, when they are manually created, Triggers have the option of either being bound to an EventLoop like BooleanEvents, or to the CommandScheduler, based on whether they recieve an EventLoop or not in the constructor. Examples on how Triggers can be used in conjunction with commands can be found [here](https://docs.wpilib.org/en/stable/docs/software/commandbased/binding-commands-to-triggers.html). 

### Getting Trigger Objects

Triggers, by their design, are very suited for usage with user inputs. As such, the WPILib library provides a collection of methods for creating Triggers from inputs, and our own `UserDigital` interface has a helper method for creating a Trigger.

#### From UserDigital

The UserDigital interface has a default method `asTrigger` which creates a Trigger from the UserDigital object. 
```java
UserDigital in = () -> true;
var trigger = in.asTrigger();
```

> Note: UserDigital also has a asBooleanEvent method which creates a BooleanEvent from the UserDigital object when provided with an EventLoop argument. There is currently no method to create a trigger with an EventLoop from a UserDigital.

#### From Digital Inputs More Generally

The command-based input classes, such as `CommandXboxController` or `CommandGenericHID` for general inputs have methods for their digital (button) inputs that create Triggers from them. As an example, to create a Trigger from the `A` button of an `XboxController`:

```java
import edu.wpi.first.wpilibj2.command.button.CommandXboxController;
var controller = new CommandXboxController(0); //input being the port on driver station
var trigger = controller.a(); // an eventLoop can also be provided
```


### The in-depth descriptions

https://docs.wpilib.org/en/stable/docs/software/commandbased/structuring-command-based-project.html

### Documentation

- https://first.wpi.edu/wpilib/allwpilib/docs/release/java/index.html
- https://api.ctr-electronics.com/phoenix/release/java/index.html
- https://codedocs.revrobotics.com/java/com/revrobotics/package-summary.html